In [1]:
import time
import os

import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool

import pandas as pd
import numpy as np

## Transfer Data

In this section, I work on developing a helper function to scrape the transfer history of a player given their name and unique ID from the website transfermarkt, using Brazilian shotstopper [Alisson](https://www.transfermarkt.us/alisson/marktwertverlauf/spieler/105470) as an example.

In [2]:
ex = 'https://www.transfermarkt.us/alisson/marktwertverlauf/spieler/105470'

In [3]:
base = 'https://www.transfermarkt.us/'
fill = '/marktwertverlauf/spieler/'
name = 'alisson'
player_id = '105470'

In [43]:
query = base + name + fill + player_id

In [19]:
query

'https://www.transfermarkt.us/alisson/marktwertverlauf/spieler/105470'

In [68]:
#%%timeit -r 1 -n 1

driver = webdriver.Chrome()
driver.get(query)
html = driver.page_source
driver.close()
sample = BeautifulSoup(html, 'html.parser') # 19.1 s

In [28]:
sample.find_all(name='button')

[<button aria-label="Close modal" class="modal__close" data-custom-close=""></button>]

In [35]:
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.implicitly_wait(0.5)
driver.get(query)

a = ActionChains(driver)
images = driver.find_elements(By.TAG_NAME,'image')
entries = []

# https://stackoverflow.com/questions/59819502/movetargetoutofboundsexception-problem-with-chromedriver-version-74
for img in images:
    #a.scroll_to_element(img).move_to_element(img).perform()
    #WebDriverWait(driver, 20).until(EC.visibility_of_element_located(img))
    #WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.TAG_NAME,'image')))
    driver.execute_script("arguments[0].scrollIntoView();", 
                          WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.TAG_NAME,'image'))))
    ActionChains(driver).move_to_element(WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.TAG_NAME,'image')))).perform()

    html = driver.page_source
    loaded = BeautifulSoup(html, 'html.parser')
    popup = loaded.find(name='div', class_='highcharts-tooltip')
    entries.append(popup.contents)

driver.close()

TimeoutException: Message: 


In [71]:
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# testing for iframe accepting privacy popup

driver = webdriver.Chrome()
driver.implicitly_wait(0.5)
driver.get(query)

wait = WebDriverWait(driver, 5)
a = ActionChains(driver)

driver.switch_to.frame(1)
accept_button = driver.find_element(By.XPATH,'//*[@id="notice"]/div[3]/div[2]/button')
a.move_to_element(wait.until(EC.element_to_be_clickable(accept_button))).click().perform()

driver.switch_to.default_content()
# html = driver.page_source
# loaded = BeautifulSoup(html, 'html.parser')
# print(loaded.prettify())
#driver.find_element(By.CLASS_NAME,'highcharts-axis-labels')

# (By.XPATH,'//*[@id="highcharts-0"]/svg/g[6]') #this XPATH doesn't work for some reason but class does?
x_axis_emt = (By.CLASS_NAME,'highcharts-axis-labels') # when x axis is visible, all images should also be in viewport
driver.execute_script("arguments[0].scrollIntoView();",
                      wait.until(EC.visibility_of_element_located(x_axis_emt)))

images = driver.find_elements(By.TAG_NAME,'image')
entries = []

for img in images:
    a.move_to_element(img).perform()
    html = driver.page_source
    loaded = BeautifulSoup(html, 'html.parser')
    info = loaded.find(name='div', class_='highcharts-tooltip')
    entries.append(info.contents)

print('done')
driver.close()

done


In [42]:
# sample code demonstrating scrolling

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.implicitly_wait(0.5)
driver.get('https://stackoverflow.com/questions/64553409/how-to-handle-google-privacy-popup-in-selenium-python')

a = ActionChains(driver)
elmt = (By.XPATH,'//*[@id="answer-64553696"]/div/div[2]/div[2]/div/div[2]/div/div[1]/a')

driver.execute_script("arguments[0].scrollIntoView();", 
                      WebDriverWait(driver, 5).until(EC.visibility_of_element_located(elmt)))
a.move_to_element(WebDriverWait(driver, 5).until(
    EC.visibility_of_element_located(elmt))).perform()

# html = driver.page_source
# loaded = BeautifulSoup(html, 'html.parser')
# popup = loaded.find(name='div', class_='highcharts-tooltip')
# entries.append(popup.contents)

driver.close()

Notes

- accept privacy now works
- run get images using 'TAG_NAME' since that was issue before